s# Lab 2
### Introduction to Machine Learning, 2021-2022 period 4

In Lab2 you will get to know an easy but powerful Machine Learning method: the k-Nearest Neighbours algorithm (k-NN). We will use the method on a classification problem, a case of supervised learning. 

This assignment is to be done with a partner. 

**Total points: 10**

**Deadline: 2022-05-20 17:00**

**Write your names and student ids here before submission: Aniek Wevers(9767002) en Redmar Leeuwendal(9665951)

## 1 The data

We look at data originally from an Australian bank. Based on some information of a customer, the bank has to decide whether this customer's application for a credit card will be approved or not.
(The original data file can be found here: https://archive.ics.uci.edu/ml/datasets/Statlog+(Australian+Credit+Approval), but for convenience we have already provided two .txt files with this assignment.)
The customer data of each customer is described by 14 variables. Four of these are categorical variables: they assign a label to each customer. These labels are represented by integers. (To protect the privacy of the customers, there is no information about the meaning of the variables and thus not about the meaning of the different labels.) Four other variables are binary (0 or 1). The other six variables are real numbers. Together they constitute the input: the vector *x*.
Each customer also has their output variable *y*, which is 0 or 1. This indicates whether the customer's application (according to human experts) should be approved (*y=1*) or not (*y=0*).

We want to write an algorithm for this task that learns from the expert decisions it has seen and is able to make its own decisions for new customers.

## 2 The algorithm

The Nearest Neighbour algorithm is based on a simple idea: we are going to base the decision for a new customer (assign credit card or not) on the decision for a customer in the training data (the historical data). We base it on the customer who is most similar to the new customer.
To use this in an algorithm, we first need a measure which can be used to determine the similarity between two customers. We will use the *Euclidean* distance for this purpose.

If we consider a customer to be a point in a 14-dimensional space (one dimension for each variable), we can calculate the Euclidean distance between two customers/points *A* and *B* with:

$$\sqrt{(A_1-B_1)^2+(A_2-B_2)^2+...+(A_{14}-B_{14})^2}$$

with $A_1$ the first input variable of $A$, $A_2$ the second, etc.

The algorithm has the following simple steps, with step 3 being an evaluation of how well the algorithm is doing:

1. Read the training data from the file;
2. For every customer $c_{test}$ in the test file: determine which customer $c_{train}$ in the training data has the smallest Euclidean distance to the test-customer $c_{test}$, and assign the decision corresponding to $c_{train}$ to $c_{test}$;
3. Count the number of test-customers for which the decision made by the algorithm is the same as the decision made by human experts;

Step 1 is already implemented below in the two classes **Data** and **Person**.
Person is a class containing all customer information: the input vector *x* and the output *y*. The inputs are divided in three different arrays: *xCategorical*, *xBinary* and *xReal*; the output is *y*.

In [ ]:
import math
import numpy as num

In [ ]:
class Person:
  def __init__(self, line_text):
    # integers
    self.xCategorical = [None]*4
    # integers
    self.xBinary = [None]*4
    # floats
    self.xReal = [None]*6

    # label (0 or 1)
    self.y = None
    # Initialize the features
    self.fromString(line_text)

  def fromString(self, line_text):
     #VARIABLES
    self.xBinary[0] = int(line_text[0])
    self.xReal[0] = float(line_text[1])
    self.xReal[1] = float(line_text[2])
    self.xCategorical[0] = int(line_text[3])
    self.xCategorical[1] = int(line_text[4])
    self.xCategorical[2] = int(line_text[5])
    self.xReal[2] = float(line_text[6])
    self.xBinary[1] = int(line_text[7])
    self.xBinary[2] = int(line_text[8])
    self.xReal[3] = float(line_text[9])
    self.xBinary[3] = int(line_text[10])
    self.xCategorical[3] = int(line_text[11])
    self.xReal[4] = float(line_text[12])
    self.xReal[5] = float(line_text[13])

  #CREDIT CARD YES OR NO (1/0)
    self.y = int(line_text[14])

  def __str__(self):
    #VARIABLES
    person_info = "x = (" + \
	    str(self.xBinary[0]) + "; " + \
	    str(self.xReal[0]) + "; " + \
	    str(self.xReal[1]) + "; " + \
	    str(self.xCategorical[0]) + "; " + \
	    str(self.xCategorical[1]) + "; " + \
	    str(self.xCategorical[2]) + "; " + \
	    str(self.xReal[2]) + "; " + \
	    str(self.xBinary[1]) + "; " + \
	    str(self.xBinary[2]) + "; " + \
	    str(self.xReal[3]) + "; " + \
	    str(self.xBinary[3]) + "; " + \
	    str(self.xCategorical[3]) + "; " + \
	    str(self.xReal[4]) + "; " + \
	    str(self.xReal[5]) + \
	    "); y = " + str(self.y) #CREDIT CARD YES/NO (1/0)
    return person_info

The class Data contains a dataset in the format of *credit_train.txt* or *credit_test.txt* (training data and test data) and uses the Person class to represent the customers.  

In [ ]:
class Data():
  def __init__(self, filename):
    self.data = []
    self.loadFile(filename)

  def loadFile(self,filename):
    with open(filename) as file:
      lines = file.readlines()
      for line in lines:
        person = Person(line.split())
        self.data.append(person)
        
  def printList(self):
    for person in self.data:
      print(person)

### A note on classes

You have not had the concept of Python classes yet in Computational Linguistics, so initially you might be confused by what is even written here. Python classes are similar to C# classes. In this case, the classes are just reading in the data and ordering it by way of named variables tied to the class name.

Suppose you have an instance of the class `Person` called `john`. If you want to know the label for `john`, you call:

`john.y`

If you have loaded the training data as an instance of the class `Data` called `trainData`, you would access the data by calling:

`trainData.data`

The same goes for methods (such as the method `printList` in `Data`). The method `__str__` in `Person` is not meant to be called by the user using that name. Rather, it is the method that gets called automatically by Python when you run:

`print(john)`

(assuming `john` is an instance of `Person`)

If you run the code below (the first time you have to run the cells above as well), you can get an idea of what the data looks like.

In [ ]:
"""
Load the train data
"""
trainData = Data("credit_train.txt")
trainData.printList()

Complete the implementation of the algorithm. 
Start with writing a function _computeDistance(person1, person2)_. 

In [ ]:
def computeDistance(person1, person2): #we calculated the distance per type of variable (xBinary,xReal,xCategorical)
    """
    This function calculates the distance between two persons from the dataset.
    The datapoints used to describe each person differentiate on three 
    types of numbers. We cannot simply subtract the datapoints of one 
    person from the datapoints of another to calculate the distance between 
    different people. Instead, this function iterates over datapoints (from person1)
    with the same type and subtracts the value of the same datapoint of person2.
    Finally it adds all of these partial distances together to get the total distance 
    between person 1 and person 2.
    """
    distancesq = 0
    for i in range(len(person1.xBinary)):
        distancesq += ((person1.xBinary[i] - person2.xBinary[i])**2)
    for i in range(len(person1.xReal)):
        distancesq += ((person1.xReal[i] - person2.xReal[i])**2)
    for i in range(len(person1.xCategorical)):
        distancesq += ((person1.xCategorical[i] - person2.xCategorical[i])**2)
    distance = math.sqrt(distancesq)
    return distance

Load the test data as well. 


In [ ]:
"""
Load the test data
"""
testData = Data("credit_test.txt")

Write a function that, given a `Person`, predicts a decision (to assign the credit card or not), only based on the `Person` in the training data that is closest.

In [ ]:
def giveCredit(person, trainData):
    """
    Predicts if a person may recieve a credit card or not based on its closest 
    neighbour from the dataset. To do this, the function compiles a sorted 
    list, consisting of the distance between person1 and all other persons in the database.
    The y-label (credit) of the most nearby person is then set as the y-label of person1.
    """
    d = []
    for i in range(len(trainData.data)):
        distance = computeDistance(person, trainData.data[i])
        d.append( (trainData.data[i].y,distance) )
    d = sorted(d, key = lambda i:i[1])
    closest_person = d[0]
    label_of_closest_person = closest_person[0]

    credit  = label_of_closest_person
    return bool(credit)

In [ ]:
giveCredit(testData.data[0], trainData)

Now write a function that determines the percentage of the test data that is wrongly classified. (Make sure that in all cases, you use the *y* from the test data only to check whether the classification of the algorithm was wrong or correct):

In [ ]:
def GetErrorPercentage(trainData, testData):
    """
    Finds the percentage of the test data that is wrongly classified 
    based on the values of the train data. To do this, the function iterates over the test data
    and increases the amount of falsely classified datapoints each time 
    the given credit in the test data does not match with the value giveCredit returns.
    This is used to calculate the percent of wrongly classified data.
    """
    total = len(testData.data)
    false = 0
    for i in range(total):
        credit = giveCredit(testData.data[i], trainData)
        if credit != bool(testData.data[i].y):
            false += 1
    percent = false/total*100
    return percent


In [ ]:
from sklearn.metrics import confusion_matrix
preds = [giveCredit(testData.data[i], trainData) for i in range(len(testData.data))]
labels = [testData.data[i].y for i in range(len(testData.data))]
confusion_matrix(labels, preds)



What percentage is wrongly classified?

In [ ]:
"""
Prints the percentage of wrongly classified data.
"""
print(GetErrorPercentage(trainData, testData))

## 2 Normalising the data
In our training data, some of the variables have very large values (up to thousands), and others have very small ones 
(like the binary variables). When you interpret the data as a set of points, this makes the training data very 'stretched' in some directions and compressed in other directions. This way, the variables with large values will dominate in the decision, whereas the smaller variables will hardly play a part. This is undesirable, because if a variable is represented in large numbers, it does not mean the variable is more important than other variables. 

This is why we are going to standardize or normalise the data: we divide all *real* variables by a number, such that the difference between the minimum and the maximum is 1. This way the real variables have the same scale as the binary variables (we leave the categorical variables for now). 
We can do this by calculating the minimum $min_i$ and maximum $max_i$ for each variable $i$, and in the formula for the Euclidean distance, for each real valued variable *i*, we replace

$$ ... + (A_i - B_i)^2 + ... $$

with

$$  ... +\left(\frac{A_i - B_i}{max_i - min_i}\right)^2 + ...$$


Write a function that determines the minimum and maximum for each real valued variable.

In [ ]:
def getRealBounds(data):
    """
    Determines the minimun and maximum for each real valued variable from the data.
    To do this, the function will create a temporary list to store all values of the
    datapoints. Next it will find the highest value and store this in another list 
    called "max". It will do this for the lowest value aswell and store this in a list
    called "min". 


    Then the function will zip these two lists together into a list called bounds
    The function will do this for each "person" in the data set.
    """
    min, max = [], []
    for i in range(len(data.data[0].xReal)):
        temporary = []
        for j in range(len(data.data)):
            temporary.append(data.data[j].xReal[i])
        max.append(max(temporary))
        min.append(min(temporary))
        bounds = list(zip(min,max))
    return bounds


In [ ]:
getRealBounds(trainData)

In [ ]:
def computeDistance2(person1, person2,getRealBounds):
    """
    This function calculates the distance between two people from the dataset.
    Different from the original computeDistance function, computeDistance2 
    calculates the distance using scaled values regarding the datapoints 
    made out of real numbers.
    """
    distancesq = 0
    for i in range(len(person1.xBinary)):
        distancesq += ((person1.xBinary[i] - person2.xBinary[i])**2)
    for i in range(len(person1.xReal)):
        distancesq += (((person1.xReal[i] - person2.xReal[i])/(getRealBounds[i][1]-getRealBounds[i][0]))**2)
    for i in range(len(person1.xCategorical)):
        distancesq += ((person1.xCategorical[i] - person2.xCategorical[i]) ** 2)
    distance = math.sqrt(distancesq)
    return distance

What percentage of the test data is wrongly classified using this distance function?

In [ ]:

def giveCredit2(person, trainData, getRealBounds):
    """
    The function giveCredit2 predicts if a person may receive a credit card or not based on its closest 
    neighbour from the dataset. The function percentage2 finds the percentage of the 
    test data that is wrongly classified, based on the values of the train data. Different 
    from the original giveCredit- and percentage function, giveCredit2 and percentage2 
    do this by using scaled values regarding the datapoints made out of real numbers.
    """
    d = []
    for i in range(len(trainData.data)):
        distance = computeDistance2(person, trainData.data[i],getRealBounds)
        d.append( (trainData.data[i].y,distance) )
    d = sorted(d, key = lambda i:i[1])
    closest_person = d[0]
    label_of_closest_person = closest_person[0]

    credit  = label_of_closest_person
    return bool(credit)

def GetErrorPercentage2(trainData, testData):
    """ Same as GetErrorPercentage but it uses giveCredit2"""
    bounds = getRealBounds(trainData)
    total = len(testData.data)
    false = 0
    for i in range(total):
        credit = giveCredit2(testData.data[i], trainData, bounds)
        if credit != bool(testData.data[i].y):
            false += 1
    percent = false/total*100
    return percent


In [ ]:
print(GetErrorPercentage2(trainData,testData))

For the categorical variables, the distance function now makes a different mistake. It assumes that points with labels 1 and 2 in a certain categorical variable are very similar but a label 14 in that variable is totally different. However, these numbers are only dummy-values that represent certain labels. For two customers, we can only say whether they are in the same category or not. The data does not say anything about the grades of similarity between categories. 

Write a third distance function _computeDistance3_ that acts the same on binary and real values as _computeDistance2_, but simply uses 0 or 1 for categorical variables instead of $...+(A_i-B_i)^2+...$. That is, it should use $0$ if $A_i = B_i$, and $1$ otherwise.

In [ ]:

def computeDistance3(person1, person2,getRealBounds):
    """
    This function calculates the distance between two persons from the dataset.
    Different from the original computeDistance2 function, computeDistance3 
    calculates the distance using scaled values regarding the datapoints 
    made out of catagorical variables. To do this, the function assigns a 0 or 1
    when it has been checked if two people from the data set are from the same
    category or not.
    """
    distancesq = 0
    for i in range(len(person1.xBinary)):
        distancesq += ((person1.xBinary[i] - person2.xBinary[i])**2)
    for i in range(len(person1.xReal)):
        distancesq += (((person1.xReal[i] - person2.xReal[i])/(getRealBounds[i][1]-getRealBounds[i][0]))**2)
    for i in range(len(person1.xCategorical)):
        if person1.xCategorical[i] == person2.xCategorical[i]:
            distancesq += 0
        else:
            distancesq += 1
    distance = math.sqrt(distancesq)
    return distance

Calculate the percentage of the test data that is wrongly classified if the algorithm uses _computeDistance3_.

In [ ]:

def giveCredit3(person, trainData, getRealBounds):
    """
    The function giveCredit3 predictis if a person may receive a credit card or not based on its closest 
    neighbour from the dataset. The function percentage3 finds the percentage of the 
    test data that is wrongly classified based on the values of the train data. Different from the original 
    giveCredit2- and percentage2 function, giveCredit3 and percentage3 do this 
    by using scaled values regarding the datapoints made out of catagorical variables.
    """
    d = []
    for i in range(len(trainData.data)):
        distance = computeDistance3(person, trainData.data[i],getRealBounds)
        d.append( (trainData.data[i].y,distance) )
    d = sorted(d, key = lambda i:i[1])
    closest_person = d[0]
    label_of_closest_person = closest_person[0]

    credit  = label_of_closest_person
    return bool(credit)

def GetErrorPercentage3(trainData, testData):
    bounds = getRealBounds(trainData)
    total = len(testData.data)
    false = 0
    for i in range(total):
        credit = giveCredit3(testData.data[i], trainData, bounds)
        if credit != bool(testData.data[i].y):
            false += 1
    percent = false/total*100
    return percent

In [ ]:
giveCredit3(testData.data[0], trainData, getRealBounds(trainData))

## 3 From NN to k-NN
The algorithm as described above is called the 1-Nearest Neighbour-algorithm. This is a special case of the *k*-Nearest Neighbours-algorithm. In the *kNN* algorithm,  we do not look at only 1 'neighbour', but the *k* points in the training data that are closest to the point we are classifying. Our new prediction is the decision that has a majority. Usually, odd numbers are used for *k*, so we do not have to worry about the case where there is an equal division. 
At *k=1* the boundary between the two areas where the algorithm approves/rejects is very uneven. By making *k* bigger the shape of the boundary is less determined by single points in the training data. This makes the boundary more smooth, which often leads to better predictions.

Modify the classification function in such a way that it uses the k-NN algorithm. Make *k* an argument of the function. You can assume that $k \in \{1,3,5\}$. Use the last distance function *computeDistance3* from above.

In [ ]:

def giveCreditk(person, trainData, getRealBounds, k):
    """
    Instead of only looking at the neighbour closest to the person we want to predict
    the credit score of, we now want to look at multiple closest neighbours. 
    The function giveCreditk will firstly compile a sorted list with the distance between the person it 
    needs to predict the approval to receive credit for and all of the other people from the dataset. 
    Then the function will take the nearest k neighbours and compute which credit score is the most common. 
    This will be the credit score the function will assign.

    The function percentagek finds the percentage of the test data that is wrongly classified based on the 
    values of the train data when looking at k closest neighbours of the point it will try to classify.
    """
    d = []
    label = []
    for i in range(len(trainData.data)):
        distance = computeDistance3(person, trainData.data[i],getRealBounds)
        d.append( (trainData.data[i].y,distance) )
    d = sorted(d, key = lambda i:i[1])
    k_closest_person = d[0:k]
    for c in k_closest_person:
        label.append(c[0])
    credit = (max(set(label), key = label.count)) #https://stackoverflow.com/questions/20038011/trying-to-find-majority-element-in-a-list
    return bool(credit)

def GetErrorPercentagek(trainData, testData, k):
    
    bounds = getRealBounds(trainData)
    total = len(testData.data)
    false = 0
    for i in range(total):
        credit = giveCreditk(testData.data[i], trainData, bounds, k)
        if credit != bool(testData.data[i].y):
            false += 1
    percent = false/total*100
    return percent

What are the test results for the different values of *k*?             

In [ ]:
"""
Calculates the percentage of the test data that is wrongly classified based on the 
values of the traindata with 1, 3 and 5 nearest neighours.
"""
k = 1
while k < 6:
    print(GetErrorPercentagek(trainData,testData,k))
    k += 2

**Note:** In a realistic situation we will not first try which value of *k* leads to the best results on the test data, but we will have to choose the value of *k* (or let the computer choose) **before** we see the test data. Usually we will do this on a separate dataset called the *development* dataset. This is an example of a general problem which we will encounter in all kinds of Machine Learning methods and on which we will pay a lot of attention in this course. 

## Congratulations! You have finished the required portion of the assignment.
### If you have finished and want to challenge yourself a little further, you can attempt the following 2 extra questions.

1) Try what happens for higher values of $k$. Discuss your results in a short paragraph.

In [ ]:

k = 1
while k < 100:
    print(GetErrorPercentagek(trainData,testData,k))
    k += 2

2) Can you think of alternate distance metrics? One possible metric is *Manhattan distance*, also called *city block* distance, and related to *lasso*, a regression analysis method that you will see later in the course. 

$dist_{MH} = \Sigma_{i=1}^{d} |A_i - B_i|$

https://en.wikipedia.org/wiki/Taxicab_geometry

Implement the manhattan distance metric. Is this a good metric for this problem? Discuss your results. 